In [3]:
from collections import defaultdict, deque
from itertools import combinations

from tqdm import tqdm
import networkx as nx
import matplotlib.pyplot as plt

In [4]:
def parse_input(file):
    with open(file) as file_in:
        wires, gates = file_in.read().split('\n\n')

    wires_starting_with_z = set()

    wire_dict = {}
    for row in wires.splitlines():
        wire, value = row.split(': ')
        wire_dict[wire] = int(value)
    
    gates_dict = {}
    for row in gates.splitlines():
        rule, out = row.split(' -> ')
        in1, op, in2 = rule.split(' ')
        gates_dict[out] = (in1, op, in2)
        if out.startswith('z'):
            wires_starting_with_z.add(out)

    return wire_dict, gates_dict, wires_starting_with_z

In [5]:
def compute_output(in1, in2, op):
    if op == 'AND':
        return in1 & in2
    elif op == 'OR':
        return in1 | in2
    elif op == 'XOR':
        return in1 ^ in2

In [6]:
def run_until_all_z_done(wires, gates, wires_starting_with_z):
    while len(wires_starting_with_z & set(wires)) != len(wires_starting_with_z):
        for out, (in1, op, in2) in gates.items():
            if in1 in wires and in2 in wires:
                wires[out] = compute_output(wires[in1], wires[in2], op)
    return wires

In [7]:
def get_wires_group_as_bin(wires, group):
    wires_order = sorted([w for w in wires if w.startswith(group)], reverse=True)
    wires_values = [str(wires[w]) for w in wires_order]
    result_bin = ''.join(wires_values)
    return result_bin

In [8]:
def compute_final_number(wires):
    result_bin = get_wires_group_as_bin(wires, 'z')
    return int(result_bin, 2)

In [29]:
def main1(file):
    wires, gates, wires_starting_with_z = parse_input(file)
    wires = run_until_all_z_done(wires, gates, wires_starting_with_z)
    return compute_final_number(wires)

In [30]:
def main2(file):
    # Solution by semi-manual diagnostic
    # Ref : https://www.uop.edu.jo/PDF%20File/petra%20university%20Digital_Design_-_A_Comprehensive_Guide_to_Digital_Electronics_and_Computer_System_Architecture-Part18.pdf
    wires, gates, wires_starting_with_z = parse_input(file)

    pairs_to_swap = [('z10', 'mkk'), ('z14', 'qbw'), ('wjb', 'cvp'), ('z34', 'wcb')]
    for x, y in pairs_to_swap:
        gates[x], gates[y] = gates[y], gates[x]

    for i, out in enumerate(sorted(wires_starting_with_z)):
        in1, op, in2 = gates[out]
        if out == 'z00':
            # First operation is just a partial sum (XOR)
            assert op == 'XOR' and set([in1, in2]) == {'x00', 'y00'}, f'Error : z{i:02d}'
        elif out == 'z01':
            # First operation is a partial sum (XOR) and the first carry (AND)
            assert op == 'XOR' and set([gates[in1][1], gates[in2][1]]) == {'AND', 'XOR'}, f'Error : z{i:02d}'
        elif out in set([f'z{i:02d}' for i in range(2, 45)]):
            # Operations 2 to n-1 consist in a partial sum (XOR) + carry of the last operation (OR)
            assert op == 'XOR', f'Error : z{i:02d}'
            assert set([gates[in1][1], gates[in2][1]]) == {'OR', 'XOR'}, f'Error : z{i:02d}'
        else:
            # Last operation is just the output carry of the n-1th addition
            assert op == 'OR' and gates[in1][1] == 'AND' and gates[in2][1] == 'AND', f'Error : z{i:02d}'

    return ','.join(sorted([x for pair in pairs_to_swap for x in pair]))

In [31]:
assert main1('example1.txt') == 4
assert main1('example2.txt') == 2024

In [32]:
main1('input.txt')

36902370467952

In [33]:
main2('input.txt')

'cvp,mkk,qbw,wcb,wjb,z10,z14,z34'